In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import molpy as mp
from pathlib import Path

data_path = Path("data/case1")

In [ ]:
struct_h = mp.Struct.from_frame(mp.io.read_pdb(data_path / "H.pdb"), "H")
struct_n = mp.Struct.from_frame(mp.io.read_pdb(data_path / "N.pdb"), "N")
struct_m = mp.Struct.from_frame(mp.io.read_pdb(data_path / "M.pdb"), "M")
struct_p = mp.Struct.from_frame(mp.io.read_pdb(data_path / "P.pdb"), "P")
struct_t = mp.Struct.from_frame(mp.io.read_pdb(data_path / "T.pdb"), "T")

typifier = mp.typifier.AmberToolsTypifier("gaff", "bcc")
typifier.typify(struct_h, workdir=data_path)
typifier.typify(struct_n, net_charge=-1, workdir=data_path)
typifier.typify(struct_m, net_charge=1, workdir=data_path)
typifier.typify(struct_p, workdir=data_path)
typifier.typify(struct_t, workdir=data_path)


In [ ]:
reacter = mp.reacter.AmberToolsReacter()
reacter.react(
    data_path,
    struct_h,
    init=struct_h.atoms[0],
    deletes=[struct_h.atoms[1], ]
)
reacter.react(struct_t, data_path)

In [ ]:
polymerizer = mp.polymerizer.AmberToolsPolymerizer()
bMIL3PEGM3 = polymerizer.linear(
    "bMIL3PEGM3",
    "HNMNMNMPPPT",
    {"H": struct_h, "N": struct_n, "M": struct_m, "P": struct_p, "T": struct_t},
    data_path,
    "HT",
)

In [ ]:
litfsi = mp.io.read_pdb(data_path / "litfsi.pdb")
litfsi = typifier.typify(litfsi, workdir=data_path)
litfsi_ff = typifier.parameterize(litfsi, workdir=data_path)

In [ ]:
packer = mp.packer.PackmolPacker(data_path/"bMIL3PEGM3_n20_litfsi_n108")
box = mp.CubeRegion(
    length=(n_atoms/density) ** (1/3),
)
packer.add_target(
    bMIL3PEGM3, num=20, region=box,
)
packer.add_target(
    litfsi, num=108, region=box,
)
frame = packer.optimize()
frame.forcefield = mp.forcefield.merge(
    bMIL3PEGM3.forcefield, litfsi_ff.forcefield
)

mp.io.write_lammps(
    data_path / "bMIL3PEGM3_n20_litfsi_n108"),
    frame
)  # data file and forcefield